# Control and audit data exploration activities with Amazon SageMaker Studio and AWS Lake Formation

This notebook accompanies the blog post "Control and audit data exploration activities with Amazon SageMaker Studio and AWS Lake Formation". The notebook demonstrates how to use SageMaker Studio along with Lake Formation to provide granular access to a data lake for different data scientists. The queries used in this notebook are based on the [Amazon Customer Reviews Dataset](https://registry.opendata.aws/amazon-reviews/), which should be registered in an existing data lake before running this code.

To compare data permissions across users, you should run the same notebook using different SageMaker user profiles.

### Prerequisites

This implementation uses Amazon Athena and the [PyAthena](https://pypi.org/project/PyAthena/) client to query data on a data lake registered with AWS Lake Formation. We will also use Pandas to run queries and store the results as Dataframes.

First we install PyAthena and import the required libraries.

In [ ]:
!pip install pyathena

In [ ]:
from pyathena import connect
import pandas as pd
import boto3

The AWS Account ID and AWS Region will be used to create an S3 bucket where Athena will save query output files. The AWS Region will also be passed as parameter when connecting to our data lake through Athena using PyAthena.

In [ ]:
sts = boto3.client("sts")
account_id = sts.get_caller_identity()["Account"]

In [ ]:
region = boto3.session.Session().region_name

In [ ]:
query_result_bucket_name = "sagemaker-audit-control-query-results-{}-{}".format(region, account_id)

### Create S3 bucket for query output files - SKIP THIS SECTION FOR THE SECOND DATA SCIENTIST USER

In [ ]:
query_result_bucket = {}

if region == "us-east-1":
    s3 = boto3.client("s3")
    query_result_bucket = s3.create_bucket(
        Bucket = query_result_bucket_name,
    )
else:
    s3 = boto3.client("s3", region_name=region)
    query_result_bucket = s3.create_bucket(
        Bucket = query_result_bucket_name,
        CreateBucketConfiguration = {
            "LocationConstraint": region
        }
    )

### Run queries using Amazon Athena and PyAthena

Once the prerequisites are configured, we can start running queries on the data lake through Athena using the PyAthena client. 

First we create a connection to Athena using PyAthena's `connect` constructor. We will pass this object as a parameter when we run queries with Pandas `read_sql` method.

In [ ]:
conn = connect(s3_staging_dir ="s3://{}/queries/".format(query_result_bucket_name), region_name=region)

Our first query will list all the databases to which this user has been granted access in the data lake.

In [ ]:
db_name_df = pd.read_sql("SHOW DATABASES", conn)
db_name = db_name_df.iloc[0][0]
print(db_name)

Our second query will list all the tables in the previous database to which this user has been granted access.

In [ ]:
tables_df = pd.read_sql("SHOW TABLES IN {}".format(db_name), conn)
table_name = tables_df.iloc[0][0]
print(table_name)

Finally we run a `SELECT` query to see all columns in the previous table to which this user has been granted access. If you have full permissions for the table, the `SELECT` query output will include the following columns:
- marketplace 
- customer_id 
- review_id 
- product_id 
- product_parent 
- product_title 
- star_rating 
- helpful_votes 
- total_votes 
- vine 
- verified_purchase 
- review_headline 
- review_body 
- review_date 
- year
- product_category

In [ ]:
df = pd.read_sql("SELECT * FROM {}.{} LIMIT 10".format(db_name, table_name), conn)
df.head(10)